![](https://raw.githubusercontent.com/UGS-CS/CSC3009/master/mashead.svg)

# <center>**Team 16**</center>

In [1]:
# import libaries
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## **1. Exploratory data analysis**

In [2]:
# Define paths
DATASET_PATH = "dataset"

CLASSES = ['glioma', 'meningioma', 'notumor', 'pituitary']

In [3]:
def eda_analysis():
    # 1. Class distribution
    train_counts = {cls: len(os.listdir(f"{DATASET_PATH}/training/{cls}"))
                   for cls in CLASSES}
    test_counts = {cls: len(os.listdir(f"{DATASET_PATH}/test/{cls}"))
                  for cls in CLASSES}

    # 2. Image size analysis
    size_stats = {}
    for split in ['training', 'test']:
        for cls in CLASSES:
            sizes = []
            for img in os.listdir(f"{DATASET_PATH}/{split}/{cls}"):
                img_path = f"{DATASET_PATH}/{split}/{cls}/{img}"
                img = tf.io.read_file(img_path)
                img = tf.image.decode_image(img)
                sizes.append(img.shape[:2])
            size_stats[f"{split}_{cls}"] = sizes

    # 3. Patient overlap check
    patient_ids = {}
    for split in ['training', 'test']:
        for cls in CLASSES:
            ids = set([f.split('_')[0] for f in os.listdir(f"{DATASET_PATH}/{split}/{cls}")])
            patient_ids[f"{split}_{cls}"] = ids

    # 4. Visualization
    plt.figure(figsize=(15,5))
    plt.subplot(1,2,1)
    sns.barplot(x=list(train_counts.keys()), y=list(train_counts.values()))
    plt.title("Training Set Distribution")

    plt.subplot(1,2,2)
    sns.barplot(x=list(test_counts.keys()), y=list(test_counts.values()))
    plt.title("Test Set Distribution")
    plt.show()

eda_analysis()

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/training/glioma'

## **2. Data preparation and pre-processing**

In [ ]:
def load_and_analyze_dataset():
    """Main EDA function for brain MRI analysis"""
    # 1. Class distribution analysis
    train_counts, test_counts = get_class_distribution()
    plot_class_distribution(train_counts, test_counts)

    # 2. Image size analysis
    size_stats = analyze_image_sizes()
    plot_size_distribution(size_stats)

    # 3. Multi-view analysis
    view_stats = analyze_image_views()
    plot_view_distribution(view_stats)

    # 4. Patient overlap check
    check_patient_overlap()

    # 5. Visual sample analysis
    visualize_samples()
